## Epigentic Association Sample Code 

Create a function to generate the linear mixed model to calculate the association between glucose and different traits   

In [ ]:
library(nlme)

Lin_Cont_DNAm_BW_LME=function(data,DNAm,random1)
{ 
  #Load the data with the trait, DNAm beta value and covariates 
  data2=na.omit(data[,c("vetrid",DNAm,"Pair","Sentrix_ID","age","smoking","bmi","med_diabetes_mellitus","cpbglu","CD8T","CD4T","NK","Bcell","Mono","Gran","dataset")])
  colnames(data2)<-c("vetrid","cg","Pair","Sentrix_ID","age","smoking","bmi","med_diabetes_mellitus","cpbglu","CD8T","CD4T","NK","Bcell","Mono","Gran","dataset")
  data2$cg<-as.numeric(as.character(data2$cg)) 
  data2<-na.omit(data2) 
  #Match the twins
  ID1<-data2[data2$dataset==1,"vetrid"]
  ID2<-data2[data2$dataset==2,"vetrid"]
  pair0_s1=na.omit(data2[data2$vetrid %in% ID1,c("Pair","cg", "vetrid")])  #If any missing value in ID1, this will produce singleton in ID2
  colnames(pair0_s1)=c("Pair","T1", "vetrid")
  pair0_s2=na.omit(data2[data2$vetrid %in% ID2,c("Pair","cg", "vetrid")])  #If any missing value in ID2, this will produce singleton in ID1
  colnames(pair0_s2)=c("Pair","T2", "vetrid")
  single1<-pair0_s1[!pair0_s1$Pair %in% pair0_s2$Pair,] #32  3
  single2<-pair0_s2[!pair0_s2$Pair %in% pair0_s1$Pair,] #0 3
  #Get paired twins and calculate mean
  pair0=merge(pair0_s1[,-3],pair0_s2[,-3],by="Pair") #unpaired samples removed! 
  dim(pair0) #131   3   75=(165-8-7)/2
  pair0$trait_mean=(pair0$T1+pair0$T2)/2 
  data11<-data2[data2$Pair %in% pair0$Pair,] #Now data11 contains only the paired subjects
  
  #Add singletons to pair0, the trait_mean is itself
  if(dim(single1)[1]>0){ 
    single1$T2<-0 
    single1$trait_mean<-single1$T1
    single1<-single1[,c("Pair", "T1", "T2", "trait_mean","vetrid")]
    pair0<-rbind(pair0, single1[,-5])
  }
  if(dim(single2)[1]>0){ 
    single2$T1<-0 
    single2$trait_mean=single2$T2
    single2<-single2[,c("Pair", "T1", "T2", "trait_mean","vetrid")]
    pair0<-rbind(pair0, single2[,-5])
  }
  single<-rbind(single1,single2)
  
  data12<-data2[data2$vetrid %in% single$vetrid,] #Now data12 contains only the unpaired subjects
  data2<-rbind(data11, data12)
  #print(dim(data2))# 294  18
  #Add pair means to phenotype data
  data2=merge(data2,pair0[,c("Pair","trait_mean")],by="Pair", all.x=T) 
  # calculate Xij-mean(xi) for each individual
  data2$trait_diff=data2$cg - data2$trait_mean
  #print(dim(data2))#164 17
  
  #print(data2[1:12,]) 
  results=rep(NA,54) #the number depends on how many cov.
  
  #in the following model, random1(Pair_ID) is nested in random2(Chip_ID)  
  fit <- formula(cpbglu~trait_diff+trait_mean+age+smoking+bmi+med_diabetes_mellitus+CD8T+CD4T+NK+Bcell+Mono+Gran)
  e=try (lme(fit, random= ~1|Sentrix_ID/Pair, data=data2, method="ML", na.action = na.omit),TRUE)
  if (class(e)[1]=="lme"){ #if LME does not converge, class(e) will be an error message. Then the calculation will be skipped 
    results[2]=n=dim(data2)[1] #number of rows, sample size 
    results[3:15]=betas=summary(e)$tTable[1:13,1]          #beta (regression coefficients)
    results[16:28]=betase=summary(e)$tTable[1:13,2]       #standard error of beta
    results[29:41]=betat=summary(e)$tTable[1:13,4]        #t stat
    results[42:54]=betap=summary(e)$tTable[1:13,5]        #p-value
  } 
  results[1]=c(DNAm)
  return(results)
} 


Load the dataset and prepare the data  

In [ ]:

## Prepare dataset for LME: merge geno, pheno and Celltype proportions
Beta0<-get(load("/sun_group/sunlab1/projects/VATwin/2019_850K/Work/beta_QN.RData"))

#get dimension of Betaa
dim(Beta0)
#846459 474


# Import annotation file, which will be used when summarizing LME results
Annot_850K<-get(load(file="/sun_group/sunlab1/projects/Yunfeng_work/InterGEN/EPIC_850K_anno.rda"))

# Import Phenotype data
pheno_Cell0=read.csv(file="/sun_group/sunlab1/projects/Students_Work/Zeyuan_work/Twin_850K/Cell_Counts_Houseman_Constrained_AllRef.csv",header=T,as.is=T) #218 8
ID <- read.csv(file="/sun_group/sunlab1/projects/Students_Work/Zeyuan_work/Twin_850K/Sample_Baseline.csv",header = T,as.is = T)
pheno_Cell1 <- merge(pheno_Cell0,ID,by.x="Sample_ID",by.y="ID_Position")

Betac<-get(load(file="/sun_group/sunlab1/projects/Students_Work/Zeyuan_work/Twin_850K/big_tot_ths1_3287cols_with4_new.rda"))  # ETS_pheno	#562 3287
#Prepare dataset and run LME, then summarize the outpu and draw QQ & Manhattan plots for each trait
pheno_V0=Betac[,c("patid","vetrid","Pair","Zyg10","age","cpbglu","smoking","bmi","med_diabetes_mellitus")] 
dim(pheno_V0) #566   9
pheno_V1=merge(pheno_V0,pheno_Cell1,by.x="vetrid",by.y="Sample_Name") 
dim(pheno_V1) #296  23 
IDs <- read.csv(file="/sun_group/sunlab1/projects/Students_Work/Zeyuan_work/Twin_850K/IDs_group.csv",header=T,as.is = T)
IDs1 <- IDs[,c("vetrid","dataset")]
pheno_V1 <- merge(pheno_V1,IDs1,by="vetrid")
#########
row.names(pheno_V1)=pheno_V1[,"Sample_ID"] 
Beta0.t <- t(Beta0)
# merge pheno & 850K DNAm Beta
data1=merge(pheno_V1,Beta0.t,by="row.names") # both patid and vetrid are available in the pheno file 
#Columns 1 through (dim(pheno_V1)[2]+1) are phenotypes, with the first column "Row.names" and the 2nd "vetrid" identical.
#Columns from (dim(pheno_V1)[2]+2) to dim(data1)[2] are DNAm; 
dim(data1) #296 846484
Number.phenos=25 #25 phenotypes
Number.cg=dim(data1)[2]-Number.phenos
Number.total=dim(data1)[2]
#pooled_sex="M_F"  
project="VETR218BW"
random1="Pair" #Specify the 1st random effect: Twin
random2="Sentrix_ID"   #Specify the 2nd random effect: Chip
cg1=colnames(data1)[(Number.phenos+1):Number.total] # Specify the methylation sites. Total number of Methyl sites specified here should equal to Number.cg
date=format(Sys.Date(), "%y%m%d")


Call the function and loop the first 10000 sites  

In [ ]:
#Call LME function which is defined in the very beginning of this program
temp.results=matrix(NA,length(cg),54)
#temp.results=matrix(NA,3,54) #for test use
cg=cg1[1:100000]# first 10000 sites; Needed to be modified for other sites calculation
#Call LME for each CpG site
for(i in 1:length(cg)){ #Do not use this line unless you are sure your code is working
  #for(i in 1:3){  # for test use
  temp.results[i,]=Lin_Cont_DNAm_BW_LME(data1,cg[i],random1)
  if (ceiling(i/10000)==(i/10000)) {print(paste("LME DNAm analysis of DNAm site #",i,"is completed."))}
} # end of for loop i
#glucose~trait_diff+trait_mean+age+smoking+medication+CD8T+CD4T+NK+Bcell+Mono+Gran
colnames(temp.results)=c("Methyl","N","Int_E","Within_E","Between_E","Age_E","Smoking_E","Bmi_E","Medication_E","CD8T_E","CD4T_E","NK_E","Bcell_E","Mono_E","Gran_E",
                         "Int_SE","Within_SE","Between_SE","Age_SE", "Smoking_SE","Bmi_SE","Medication_SE","CD8T_SE","CD4T_SE","NK_SE","Bcell_SE","Mono_SE","Gran_SE",
                         "Int_t","Within_t","Between_t","Age_t","Smoking_t","Bmi_t","Medication_t","CD8T_t","CD4T_t","NK_t","Bcell_t","Mono_t","Gran_t",
                         "Int_P","Within_P","Between_P", "Age_P","Smoking_P","Bmi_t","Medication_t","CD8T_P","CD4T_P","NK_P","Bcell_P","Mono_P","Gran_P")  

#Save the output
output = "/sun_group/sunlab1/projects/Students_Work/Yijin_work/glucose/LME_Twin850k_dm_sub1.rda"
save(temp.results,file=output)

Create a SH file for bash  

In [ ]:
# Tell the server to access the code and where to store the report of running the code
#!/bin/sh 
R CMD BATCH /sun_group/sunlab1/projects/Students_Work/Yijin_work/glucose/glucose_10000.R 
/sun_group/sunlab1/projects/Students_Work/Yijin_work/glucose/glucose_10000.Rout

Store the SH file and Code file on the server; Run the code through terminal  

In [ ]:
qsub -cwd -q sunlab.q glucose_10000.sh
qsub -cwd -q sunlab.q glucose_20000.sh
qsub -cwd -q sunlab.q glucose_30000.sh
qsub -cwd -q sunlab.q glucose_40000.sh
qsub -cwd -q sunlab.q glucose_50000.sh
qsub -cwd -q sunlab.q glucose_60000.sh
qsub -cwd -q sunlab.q glucose_70000.sh
qsub -cwd -q sunlab.q glucose_80000.sh
qsub -cwd -q sunlab.q glucose_90000.sh
qstat -u "*" #Code to check the status of submitted files